In [26]:
pip install yfinance backtrader ta scipy

In [40]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta
import backtrader as bt
from scipy.optimize import brute
import matplotlib.pyplot as plt


In [41]:
%matplotlib inline

In [42]:
def fetch_data(symbol, start, end):
    data = yf.download(symbol, start=start, end=end)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.droplevel(level=1)
    return data

# Fetching data for TCS for a period of 4 years
data = fetch_data('TCS.NS', '2019-01-01', '2023-12-31')

data.head(10)


[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2019-01-01 00:00:00+00:00,1712.980835,1902.800049,1910.000000,1885.000000,1896.000000,1094883
2019-01-02 00:00:00+00:00,1731.435425,1923.300049,1934.449951,1900.000000,1905.000000,2100463
2019-01-03 00:00:00+00:00,1710.414307,1899.949951,1944.949951,1893.099976,1919.000000,2611668
2019-01-04 00:00:00+00:00,1689.619141,1876.849976,1901.199951,1841.000000,1900.000000,4280862
2019-01-07 00:00:00+00:00,1708.569336,1897.900024,1908.800049,1881.000000,1891.800049,1856423
2019-01-08 00:00:00+00:00,1704.653564,1893.550049,1906.400024,1883.300049,1905.400024,1691756
2019-01-09 00:00:00+00:00,1698.711792,1886.949951,1919.000000,1866.699951,1907.400024,2414376
2019-01-10 00:00:00+00:00,1700.152100,1888.550049,1905.000000,1872.099976,1890.000000,3053461
2019-01-11 00:00:00+00:00,1658.740845,1842.550049,1875.000000,1835.000000,1870.000000,9209862


In [43]:
class SMAStrategyWithRisk(bt.Strategy):
    params = (
        ('sma1', 50),
        ('sma2', 200),
        ('stop_loss', 0.02),  # 2% stop-loss
        ('take_profit', 0.05),  # 5% take-profit
    )

    def __init__(self):
        self.sma1 = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.sma1)
        self.sma2 = bt.indicators.SimpleMovingAverage(
            self.data.close, period=self.params.sma2)
        self.order = None
        self.buy_price = None
        self.stop_loss_marker = []
        self.take_profit_marker = []

    def next(self):
        if self.order:
            return

        if self.sma1[0] > self.sma2[0]:
            if not self.position:
                self.buy_price = self.data.close[0]
                self.order = self.buy()
        elif self.sma1[0] < self.sma2[0]:
            if self.position:
                self.order = self.sell()

        if self.position:
            if self.position.size > 0:
                stop_price = self.buy_price * (1.0 - self.params.stop_loss)
                take_profit_price = self.buy_price * (1.0 + self.params.take_profit)
                if self.data.close[0] <= stop_price:
                    self.order = self.sell()
                    self.stop_loss_marker.append(self.data.datetime.date(0))
                elif self.data.close[0] >= take_profit_price:
                    self.order = self.sell()
                    self.take_profit_marker.append(self.data.datetime.date(0))

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buy_price = order.executed.price

    def plot_markers(self):
        plt.plot(self.data.close)
        for date in self.stop_loss_marker:
            plt.axvline(date, color='red', linestyle='--', lw=2, label='Stop Loss')
        for date in self.take_profit_marker:
            plt.axvline(date, color='green', linestyle='--', lw=2, label='Take Profit')
        plt.legend()


In [44]:
def run_backtest(sma1, sma2):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(SMAStrategyWithRisk, sma1=sma1, sma2=sma2)
    data_bt = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(data_bt)
    cerebro.broker.set_cash(1000000)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.run()
    return cerebro.broker.getvalue()

def optimize_strategy():
    def objective(params):
        sma1, sma2 = params
        return -run_backtest(sma1, sma2)  # Negative for minimization

    ranges = (slice(10, 100, 10), slice(100, 300, 10))
    result = brute(objective, ranges, finish=None)
    return result

optimal_params = optimize_strategy()
print(f"Optimal SMA1: {optimal_params[0]}, Optimal SMA2: {optimal_params[1]}")


Optimal SMA1: 30.0, Optimal SMA2: 190.0


In [45]:
cerebro = bt.Cerebro()
cerebro.addstrategy(SMAStrategyWithRisk, sma1=int(optimal_params[0]), sma2=int(optimal_params[1]))
data_bt = bt.feeds.PandasData(dataname=data)
cerebro.adddata(data_bt)
cerebro.broker.set_cash(1000000)
cerebro.addsizer(bt.sizers.FixedSize, stake=10)
results = cerebro.run()

# Get the strategy instance
strategy = results[0]

# Plot the backtest result with candlestick style
cerebro.plot(style='candlestick')

# Plot stop-loss and take-profit markers
plt.figure(figsize=(10, 5))
plt.plot(data['Close'], label='Close Price')
strategy.plot_markers()
plt.title('Stop Loss and Take Profit Markers')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
plt.figure(figsize=(10, 5))
plt.plot(data['Close'], label='Close Price')
strategy.plot_markers()
plt.title('Stop Loss and Take Profit Markers')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>